In [1]:
# 2.1
# Создайте кастомный класс датасета для работы с CSV файлами:
# - Загрузка данных из файла
# - Предобработка (нормализация, кодирование категорий)
# - Поддержка различных форматов данных (категориальные, числовые, бинарные и т.д.)
import torch
import pandas as pd
from torch.utils.data import Dataset


class CSVDataset(Dataset):
    def __init__(self, file_path, numeric_cols=None, categorical_cols=None, binary_cols=None, target_col=None):
        """
        Args:
            file_path (str): Путь к CSV файлу
            numeric_cols (list): Список числовых колонок
            categorical_cols (list): Список категориальных колонок
            binary_cols (list): Список бинарных колонок
            target_col (str): Искомое
        """
        self.data = pd.read_csv(file_path)
        self.numeric_cols = numeric_cols if numeric_cols else []
        self.categorical_cols = categorical_cols if categorical_cols else []
        self.binary_cols = binary_cols if binary_cols else []
        self.target_col = target_col

        self.preprocess_data()

    def preprocess_data(self):
        # Нормализация числовых данных (MinMax)
        for col in self.numeric_cols:
            col_data = torch.tensor(self.data[col].values, dtype=torch.float32)
            min_val = torch.min(torch.tensor(col_data))
            max_val = torch.max(torch.tensor(col_data))
            self.data[col] = ((col_data - min_val) / (max_val - min_val)) - 1

        # Кодирование категориальных данных (One-Hot), громоздкое но универсальное
        for col in self.categorical_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)
            # Конвертируем в one-hot
            one_hot = torch.zeros((len(self.data), len(unique_values)))
            one_hot[torch.arange(len(self.data)), self.data[col].values] = 1
            # Удаляем оригинальную колонку и добавляем one-hot колонки
            self.data.drop(col, axis=1, inplace=True)
            for i, val in enumerate(unique_values):
                self.data[f"{col}_{val}"] = one_hot[:, i]

        # Кодирование бинарных данных
        for col in self.binary_cols:
            unique_values = self.data[col].unique()
            mapping = {v: i for i, v in enumerate(unique_values)}
            self.data[col] = self.data[col].map(mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = []

        # Собираем все признаки и объеденяем в один тензор с которым сможет работать Pytorch
        for col in self.numeric_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.binary_cols:
            features.append(torch.tensor(self.data.iloc[idx][col], dtype=torch.float32))

        for col in self.categorical_cols:
            for c in self.data.columns:
                if c.startswith(f"{col}_"):
                    features.append(torch.tensor(self.data.iloc[idx][c], dtype=torch.float32))

        features = torch.stack(features)

        if self.target_col:
            target = torch.tensor(self.data.iloc[idx][self.target_col], dtype=torch.float32)
            return features, target


        return features

In [ ]:
Класс выше, загружаает при помощи pandas csv файл, затем проводит предобработку, нормализуя численные признаки, и кодируя бинарные и категориальные,
по итогу весь датасет преобразуется в тензор pytorch в формате float32

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import mse, log_epoch


class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)


def start_train(dataset, in_features=1):
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')

    # Создаём модель, функцию потерь и оптимизатор
    model = LinearRegression(in_features)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # Параметры регуляризации
    l_lambda = 0.01  # коэффициент для регуляризации
    regularization = 'l2'  # тип регуляризации l1, l2 или None

    # Параметры early stopping
    patience = 5  # количество эпох без улучшения перед остановкой
    best_loss = float('inf')
    epochs_without_improvement = 0

    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        total_loss = 0

        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred = model(batch_X)

            # Основная функция потерь
            loss = criterion(y_pred, batch_y)

            # L2
            if regularization == 'l2':
                l2_reg = torch.tensor(0.)
                for param in model.parameters():
                    l2_reg += torch.norm(param, p=2) ** 2
                loss += l_lambda * l2_reg
            # L1
            elif regularization == 'l1':
                l1_reg = torch.tensor(0.)
                for param in model.parameters():
                    l1_reg += torch.norm(param, p=1)
                loss += l_lambda * l1_reg

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (i + 1)

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            epochs_without_improvement = 0
            # Сохраняем лучшую модель
            torch.save(model.state_dict(), 'models/best_linreg_torch.pth')
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Early stopping на эпохе {epoch}')
                break

        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss)

def evaluate():
    best_model = LinearRegression(in_features=1)
    best_model.load_state_dict(torch.load('models/best_linreg_torch.pth'))
    best_model.eval()

In [3]:
# 2.2
dataset = CSVDataset("data/Salary_dataset.csv", numeric_cols = ["YearsExperience"], target_col = "Salary") # Простой датасет с одним числовым признаком - https://www.kaggle.com/datasets/abhishek14398/salary-dataset-simple-linear-regression
start_train(dataset,1)
dataset = CSVDataset("data/Multiple.csv", numeric_cols = ["age","experience"], target_col = "income") # чуть более сложный датасет, с двумя числовыми признаками - https://www.kaggle.com/datasets/hussainnasirkhan/multiple-linear-regression-dataset
start_train(dataset,2)


C:\Users\MobS\AppData\Local\Temp\ipykernel_11236\234280877.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_val = torch.min(torch.tensor(col_data))
C:\Users\MobS\AppData\Local\Temp\ipykernel_11236\234280877.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_val = torch.max(torch.tensor(col_data))


Размер датасета: 30
Количество батчей: 1


C:\Users\MobS\mambaforge\envs\main\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10: loss=868343360.0000
Epoch 20: loss=830120576.0000
Epoch 30: loss=817326656.0000
Epoch 40: loss=807955904.0000
Epoch 50: loss=801070016.0000
Epoch 60: loss=796010112.0000
Epoch 70: loss=792292032.0000
Epoch 80: loss=789559744.0000
Epoch 90: loss=787552064.0000
Epoch 100: loss=786076800.0000
Размер датасета: 20
Количество батчей: 1


C:\Users\MobS\AppData\Local\Temp\ipykernel_11236\234280877.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_val = torch.min(torch.tensor(col_data))
C:\Users\MobS\AppData\Local\Temp\ipykernel_11236\234280877.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_val = torch.max(torch.tensor(col_data))
C:\Users\MobS\mambaforge\envs\main\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10: loss=114498128.0000
Epoch 20: loss=105904104.0000
Epoch 30: loss=99983072.0000
Epoch 40: loss=95619784.0000
Epoch 50: loss=92401680.0000
Epoch 60: loss=90025824.0000
Epoch 70: loss=88269776.0000
Epoch 80: loss=86970104.0000
Epoch 90: loss=86006688.0000
Epoch 100: loss=85291264.0000


Проверил работосопособность CSVDataset и тренировки линейной регрессии на небольших датасетах с kaggle, ссылка на каждый в комментариях к коду выше. отдельно проверял работу кодировки признаков, но не могу показать их в этой работе, т.к. не смог найти датасет который имел в себе категориальные или бинарные признаки, и при этом не был бы гигантским, из за чего его обучение занимало очень много времени и при этом активировало раннюю остановку на 7-ой эпохе, то есть эффективное обучение шло 2-е эпохи, что невероятно мало, но сам факт успешного начала обучения доказывает работоспособность кодировки.